In [1]:
import torch
import os
from transformers import AutoTokenizer, LEDConfig, LEDForConditionalGeneration

src_text = [

    """ PG&E stated it scheduled the blackouts in response to forecasts for high winds amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."""
]

device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "../models/primera-multi_lexsum-source-short"
tokenizer = AutoTokenizer.from_pretrained(model_name)
config = LEDConfig.from_pretrained(model_name)
model = LEDForConditionalGeneration.from_pretrained(model_name).to(device)

/home/keddie/anaconda3/envs/facilex_caselaw/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/home/keddie/anaconda3/envs/facilex_caselaw/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:471: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `0` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [2]:
from datasets import load_dataset
# please install HuggingFace datasets by pip install datasets 

multi_lexsum = load_dataset("allenai/multi_lexsum", name="v20220616")
# Download multi_lexsum locally and load it as a Dataset object 

example = multi_lexsum["validation"][0] # The first instance of the dev set 

for sum_len in ["long", "short", "tiny"]:
    print(example["summary/" + sum_len]) # Summaries of three lengths

On January 23, 2004, Plaintiff filed an amended complaint under Title VII of the Civil Rights Act of 1964 and the Pregnancy Discrimination Act, 42 U.S.C. § 2000e et seq. and 2000e(k), against AT&T Corporation in the United States District Court for the Western District of Missouri.  The plaintiff, represented by private counsel, was a former AT&T employee and asked the Court for declaratory and injunctive relief, as well as damages, alleging that AT&T's health insurance policy discriminated against women.  Specifically, the plaintiff contended that the defendant's health insurance plan, which did not provide prescription contraceptives (birth control) before 2002 and only through the mail after 2002, violated female employees' civil rights.

The action originally started in the U.S. District Court for the District of Kansas, but was transferred to Missouri on May 12, 2003.  This is the date on which the docket begins.  Originally, there were two plaintiffs on the case, but one dropped 

In [3]:
test_data = multi_lexsum["test"][0]["sources"]

print(test_data)
# multi_lexsum["test"][0]

['Case 2:14-cv-00186-MEF-CSC Document 1 Filed 03/18/14 Page 1 of 24\n\nRECEiVED\nIN THE UNITED STATES DIscTcquR FOR THE MIDDLE DISTRICTLO ALAAMA 1\n\nSHARNALLE MITCHELL, LORENZO BROWN,\n\nDEB -RA P. HCETT. CLV\n\nUS u 1 STCid\n\n3 ALA flCTr\'T\nI Li Li L i\n\n4\n\nCOURTNEY TUBBS,\n\nTITO WILLIAMS,\n\nPlaintiffs,\n\nV.\nTHE CITY OF MONTGOMERY,\n\nCase No. _________\n\nDefendant.\n\nCOMPLAINT Introduction The Plaintiffs in this case are each impowrished people who were jailed by the City of Montgomery because they were unable to pay a debt owed to the City from traffic tickets. In each case, the City ordered the Plamtiff either to pay the City immediately or to "sit out" his or her debt in the City jail at a rate of $50 per day. Although the Plaintiffs pleaded that they were Unable to pay due to their poverty, each was sent to jail for nonpayment and none was afforded the inquiry into their ability to pay that the United States and Alabama Constitutions require. Once locked in the City\'

In [5]:
batch = tokenizer(test_data, truncation=True, padding="longest", return_tensors="pt").to(device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)

print(tgt_text)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
